# 모듈 불러오기

In [49]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import ast
import copy
import numpy as np

import redis
from redisgraph import Node, Edge, Graph, Path

from pyvis.network import Network

%matplotlib inline

# redisgraph database 연결

In [2]:
# RedisGraph Instance 생성
r = redis.Redis(host='127.0.0.1', port=6379, db=1)

In [3]:
purchase_graph = Graph('product_recommendation', r)

# product recommendation based on redisgraph

In [4]:
def columns_select(result):
    """
    쿼리 실행 결과의 header값을 가져오는 함수 
    
    params
    - result: query 실행 결과 (class QueryResult)
    """
    columns = []
    
    for i in range(len(result.header)):
        col = result.header[i][1].decode('utf-8')
        columns.append(col)
    
    return columns

In [115]:
def top_product_list(query_input):
    """
    테이블 형태로 추천 상품의 리스트를 반환하는 함수 
    
    parmas
    - query_input: 입력 쿼리
    """
    result = purchase_graph.query(query_input)
    cols = columns_select(result)
    
    top_prod = pd.DataFrame(result.result_set, columns = cols)
    
    return top_prod

## 많이 본 상품 Top 5

In [116]:
query_view_top5 = """match (:customer)-[v:view]->(p:product)
                    return p.id as product_id, p.name as product_sku, count(v) as view_cnt 
                    order by view_cnt desc limit 5;"""

top5_view_prod = top_product_list(query_view_top5)
top5_view_prod

,product_id,product_sku,view_cnt
0,183,[145],294
1,23,[49],288
2,4,[159],286
3,152,[100],285
4,89,[63],283


## 많이 구매한 상품 Top 5

In [117]:
query_trans_top5 = """match (:order)-[c:contain]->(p:product)
                    return p.id as product_id, p.name as product_sku, count(c) as trans_cnt 
                    order by trans_cnt desc limit 5;"""

top5_trans_prod = top_product_list(query_trans_top5)
top5_trans_prod

,product_id,product_sku,trans_cnt
0,25,[70],5
1,145,[88],4
2,37,[45],4
3,150,[129],4
4,64,[33],4


## 특정 상품을 본 사람들이 본 다른 상품 추천

In [118]:
def recommendation_prod(query_input):
    """
    다른 상품을 구매하거나 본 경우와 관련된 추천 상품을 
    Table형태로 반환하는 함수 
    
    parmas
    - query_input: 입력 쿼리
    """
    result = purchase_graph.query(query_input)
    cols = columns_select(result)
    
    t_df = pd.DataFrame(result.result_set, columns = cols)
    
    rec_prod_id = t_df.rec_prod_node.apply(lambda x: ast.literal_eval(x)['properties']['id'])
    rec_prod_nm = t_df.rec_prod_node.apply(lambda x: ast.literal_eval(x)['properties']['name'])
    
    rec_prod = pd.concat([rec_prod_id, rec_prod_nm, t_df['count']], axis=1)
    rec_prod.columns = ['product_id', 'product_sku', 'count']

    return rec_prod

### Table format

In [11]:
# 가장 많이 본 상품

top5_view_prod.iloc[0]

product_id       183
product_sku    [145]
view_cnt         294
Name: 0, dtype: object

In [119]:
query_rec_topview = """match (p:product {id: 183})<-[:view]-(u:customer)-[v:view]->(rec_prod:product) 
                        where not rec_prod.id = 183 
                        return toJson(rec_prod) as rec_prod_node, count(v) as count 
                        order by count desc limit 5;"""

rec_topview = recommendation_prod(query_rec_topview)
rec_topview

,product_id,product_sku,count
0,64,[33],21
1,15,[37],21
2,44,[79],17
3,5,[169],17
4,192,[83],16


### Graph format

In [42]:
query_rec_topview = """match (p:product {id: 183})<-[:view]-(:customer)-[v:view]->(rec_prod:product) 
                        where not rec_prod.id = 183 
                        with rec_prod, count(v) as count order by count desc limit 5 
                        match (rec_prod)<-[rec_v:view]-(u:customer)-[origin_v:view]->(p:product {id:183}) 
                        return toJson(collect(distinct rec_prod)) as rec_prod,
                        toJson(collect(distinct rec_v)) as rec_view, 
                        toJson(collect(distinct u)) as rel_user,
                        toJson(collect(distinct origin_v)) as origin_view,
                        toJson(collect(distinct p)) as view_prod;"""

result_rec_topview = purchase_graph.query(query_rec_topview)

In [43]:
records = result_rec_topview.result_set[0]

In [48]:
rec_prod = ast.literal_eval(records[0])
edges_1st = ast.literal_eval(records[1])
users = ast.literal_eval(records[2])
edges_2nd = ast.literal_eval(records[3])
view_prod = ast.literal_eval(records[4])

In [77]:
network1 = Network(height="750px", width="100%", font_color="black", 
                   heading="Recommendation for users who view the specific product", directed = True, notebook=True)

In [78]:
network1.barnes_hut(gravity=-7000, central_gravity=0.02, spring_length=80, damping=0.05, overlap=0.50)

In [79]:
for node in rec_prod:
    network1.add_node(node['id'], label = node['properties']['name'], 
                     title = str(node['labels'])+': sku '+str(node['properties']['name']), color = '#771DAE')

for node in users:
    network1.add_node(node['id'], label = node['properties']['name'], 
                     title = str(node['labels'])+': sku '+str(node['properties']['name']), color = '#949494')
    
for node in view_prod:
    network1.add_node(node['id'], label = node['properties']['name'], 
                     title = str(node['labels'])+': sku '+str(node['properties']['name']), color = '#B8B5D2')

In [80]:
for edge in edges_1st:
    network1.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_2nd:
    network1.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])

In [81]:
network1.show('./graph/Recommendation_view_one_prod.html')

## 특정 상품을 구매한 사람들이 구매한 다른 상품 추천

### Table format

In [82]:
# 가장 많이 구매한 상품

top5_trans_prod.iloc[0]

product_id       25
product_sku    [70]
trans_cnt         5
Name: 0, dtype: object

In [121]:
query_rec_toptrans = """match (p:product {id: 25})<-[c1:contain]-(o1:order)<-[t1:transact]-(u:customer)
                        with p, c1, o1, t1,u
                        match (u)-[t2:transact]->(o2:order)-[c2: contain]->(rec_prod:product)
                        where not rec_prod.id = 25
                        return toJson(rec_prod) as rec_prod_node, count(c2) as count 
                        order by count desc limit 5;"""


rec_toptrans = recommendation_prod(query_rec_toptrans)
rec_toptrans

,product_id,product_sku,count
0,43,[67],1


### Graph format

In [101]:
query_rec_toptrans = """match (p:product {id: 25})<-[c1:contain]-(o1:order)<-[t1:transact]-(u:customer)
                        with p, c1, o1, t1,u
                        match (u)-[t2:transact]->(o2:order)-[c2: contain]->(rec_prod:product)
                        where not rec_prod.id = 25
                        with rec_prod, count(c2) as transact_num order by transact_num desc limit 5
                        match (rec_prod)<-[c2: contain]-(o2:order)<-[t2:transact]-(u:customer)-[t1:transact]->(o1:order)-[c1:contain]->(p:product {id: 25})
                        return toJson(collect(distinct rec_prod)) as rec_prod,
                        toJson(collect(distinct c2)) as edges_1st, 
                        toJson(collect(distinct o2)) as order_rec_prod,
                        toJson(collect(distinct t2)) as edges_2nd,
                        toJson(collect(distinct u)) as users,
                        toJson(collect(distinct t1)) as edges_3rd,
                        toJson(collect(distinct o1)) as order_spec_prod,
                        toJson(collect(distinct c1)) as edges_4th,
                        toJson(collect(distinct p)) as spec_prod;"""

result_rec_toptrans = purchase_graph.query(query_rec_toptrans)

In [106]:
records = result_rec_toptrans.result_set[0]

In [107]:
rec_prod = ast.literal_eval(records[0])
edges_1st = ast.literal_eval(records[1])
order_rec_prod = ast.literal_eval(records[2])
edges_2nd = ast.literal_eval(records[3])
users = ast.literal_eval(records[4])
edges_3rd = ast.literal_eval(records[5])
order_spec_prod = ast.literal_eval(records[6])
edges_4th = ast.literal_eval(records[7])
spec_prod = ast.literal_eval(records[8])

In [108]:
network2 = Network(height="750px", width="100%", font_color="black", 
                   heading="Recommendation for users who purchase the specific product", directed = True, notebook=True)

In [109]:
network2.barnes_hut(gravity=-7000, central_gravity=0.02, spring_length=80, damping=0.05, overlap=0.50)

In [110]:
for node in rec_prod:
    network2.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#771DAE')

for node in order_rec_prod:
    network2.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#949494')
    
for node in users:
    network2.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#B8B5D2')

for node in order_spec_prod:
    network2.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#949494')
    
for node in spec_prod:
    network2.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#F30259')

In [112]:
for edge in edges_1st:
    network2.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_2nd:
    network2.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_3rd:
    network2.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_4th:
    network2.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])

In [114]:
network2.show('./graph/Recommendation_purchase_one_prod.html')

## 특정 상품을 본 사람들이 구매한 다른 상품 추천

### Table format

In [122]:
query_rec_v_p = """match (p:product {id: 183})<-[:view]-(u:customer)-[t:transact]->(o:order)-[c:contain]->(rec_prod:product) 
                    where not rec_prod.id = 183 
                    return toJson(rec_prod) as rec_prod_node, count(c) as count 
                    order by count desc limit 5;"""

In [123]:
rec_v_top_p = recommendation_prod(query_rec_v_p)
rec_v_top_p

,product_id,product_sku,count
0,189,[113],1
1,42,[64],1
2,192,[83],1
3,75,[186],1
4,138,[82],1


### Graph format

In [126]:
query_rec_v_p = """match (p:product {id: 183})<-[:view]-(u:customer)-[t:transact]->(o:order)-[c:contain]->(rec_prod:product)
                    where not rec_prod.id = 183 
                    with rec_prod, count(c) as count order by count desc limit 5 
                    match (rec_prod)<-[c:contain]-(o:order)-[t:transact]-(u:customer)-[v:view]->(p:product {id:183}) 
                    return toJson(collect(distinct rec_prod)) as rec_prod,
                    toJson(collect(distinct c)) as edges_1st, 
                    toJson(collect(distinct o)) as rec_order,
                    toJson(collect(distinct t)) as edges_2nd,
                    toJson(collect(distinct u)) as users,
                    toJson(collect(distinct v)) as edges_3rd,
                    toJson(collect(distinct p)) as spec_prod;"""

result_rec_v_top_p = purchase_graph.query(query_rec_v_p)

In [127]:
records = result_rec_v_top_p.result_set[0]

In [128]:
rec_prod = ast.literal_eval(records[0])
edges_1st = ast.literal_eval(records[1])
rec_order = ast.literal_eval(records[2])
edges_2nd = ast.literal_eval(records[3])
users = ast.literal_eval(records[4])
edges_3rd = ast.literal_eval(records[5])
spec_prod = ast.literal_eval(records[6])

In [129]:
network3 = Network(height="750px", width="100%", font_color="black", 
                   heading="Recommendation for users who view the specific product", directed = True, notebook=True)

In [130]:
network3.barnes_hut(gravity=-7000, central_gravity=0.02, spring_length=80, damping=0.05, overlap=0.50)

In [135]:
for node in rec_prod:
    network3.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#771DAE')

for node in rec_order:
    network3.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#949494')
    
for node in users:
    network3.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#B8B5D2')
    
for node in spec_prod:
    network3.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#F30259')

In [136]:
for edge in edges_1st:
    network3.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_2nd:
    network3.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_3rd:
    network3.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])

In [137]:
network3.show('./graph/Recommendation_view_one_purchase_prod.html')

## 특정 상품을 구매한 사람들이 본 다른 상품

### Table format

In [138]:
query_rec_p_v = """match (p:product {id: 25})<-[c1:contain]-(o1:order)<-[t1:transact]-(u:customer)
                    with p, c1, o1, t1,u
                    match (u)-[v: view]->(rec_prod:product)
                    where not rec_prod.id = 25
                    return toJson(rec_prod) as rec_prod_node, count(v) as count 
                    order by count desc limit 5;"""

rec_p_top_v = recommendation_prod(query_rec_p_v)
rec_p_top_v

,product_id,product_sku,count
0,134,[110],2
1,8,[36],2
2,146,[194],2
3,106,[9],1
4,186,[43],1


### Graph format

In [141]:
query_rec_rec_p_v  = """match (p:product {id: 25})<-[c1:contain]-(o1:order)<-[t1:transact]-(u:customer)
                        with p, c1, o1, t1,u
                        match (u)-[v: view]->(rec_prod:product)
                        where not rec_prod.id = 25
                        with rec_prod, count(v) as count order by count desc limit 5
                        match (rec_prod)<-[v: view]-(u:customer)-[t1:transact]->(o1:order)-[c1:contain]->(p:product {id: 25})
                        return toJson(collect(distinct rec_prod)) as rec_prod,
                        toJson(collect(distinct v)) as edges_1st, 
                        toJson(collect(distinct u)) as users,
                        toJson(collect(distinct t1)) as edges_2nd,
                        toJson(collect(distinct o1)) as order_spec_prod,
                        toJson(collect(distinct c1)) as edges_3rd,
                        toJson(collect(distinct p)) as spec_prod;"""

result_rec_p_v = purchase_graph.query(query_rec_rec_p_v )

In [140]:
records = result_rec_p_v.result_set[0]

In [142]:
rec_prod = ast.literal_eval(records[0])
edges_1st = ast.literal_eval(records[1])
users = ast.literal_eval(records[2])
edges_2nd = ast.literal_eval(records[3])
order_spec_prod = ast.literal_eval(records[4])
edges_3rd = ast.literal_eval(records[5])
spec_prod = ast.literal_eval(records[6])

In [149]:
network4 = Network(height="750px", width="100%", font_color="black", 
                   heading="Recommendation for users who purchase the specific product", directed = True, notebook=True)

In [150]:
network4.barnes_hut(gravity=-7000, central_gravity=0.02, spring_length=80, damping=0.05, overlap=0.50)

In [151]:
for node in rec_prod:
    network4.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#771DAE')

for node in users:
    network4.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#B8B5D2')
    
for node in order_spec_prod:
    network4.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#949494')
    
for node in spec_prod:
    network4.add_node(node['id'], label = node['properties']['name'], 
                     title = node['labels'], color = '#F30259')

In [152]:
for edge in edges_1st:
    network4.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_2nd:
    network4.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])
    
for edge in edges_3rd:
    network4.add_edge(edge['start']['id'], edge['end']['id'], title = edge['relationship'])

In [153]:
network4.show('./graph/Recommendation_purchase_one_view_prod.html')